# Group 1- Google Play App Store Data Analysis

## Download the data from S3

In [1]:
!wget https://gwu-mileypiao888.s3.amazonaws.com/Google_Playstore.csv

--2019-12-02 00:23:13--  https://gwu-mileypiao888.s3.amazonaws.com/Google_Playstore.csv
Resolving gwu-mileypiao888.s3.amazonaws.com (gwu-mileypiao888.s3.amazonaws.com)... 52.216.170.227
Connecting to gwu-mileypiao888.s3.amazonaws.com (gwu-mileypiao888.s3.amazonaws.com)|52.216.170.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31416840 (30M) [text/csv]
Saving to: ‘Google_Playstore.csv’

Google_Playstore.cs 100%[===================>]  29.96M  52.7MB/s    in 0.6s    

2019-12-02 00:23:14 (52.7 MB/s) - ‘Google_Playstore.csv’ saved [31416840/31416840]



In [1]:
!pip freeze | grep -E 'ipython-sql|psycopg2'

You are using pip version 18.0, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
ipython-sql==0.3.9
psycopg2==2.7.5
psycopg2-binary==2.7.5


In [3]:
!dropdb -U student Group1

dropdb: database removal failed: ERROR:  database "Group1" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [4]:
!createdb -U student Group1

createdb: database creation failed: ERROR:  database "Group1" already exists


In [2]:
%load_ext sql

In [3]:
%sql postgresql://student@/Group1

'Connected: student@Group1'

## Create original table

Original table is used to create fact and dimension tables.

In [4]:
%%sql
DROP TABLE IF EXISTS Google_Playstore Cascade;

CREATE TABLE Google_Playstore (
APP_ID NUMERIC(10) NOT NULL,
APP_NAME VARCHAR(100),
CATE_NAME VARCHAR(50),
RATING NUMERIC(3,2),
REVIEW NUMERIC(25),
INSTALL NUMERIC(10),
SIZE_NUMBER NUMERIC(10,2),
SIZE_MAGNITUDE VARCHAR(10),
FULL_SIZE VARCHAR(10), 
PRICE NUMERIC(10,2),
CONTENT_RATING VARCHAR(25),
LAST_UPDATE_DAY VARCHAR(3),
LAST_UPDATE_MONTH VARCHAR(25),
LAST_UPDATE_YEAR VARCHAR(4),
FULL_DATE VARCHAR(15),
MIN_VERSION VARCHAR(25),
LATEST_VERSION VARCHAR(100),
    
    PRIMARY KEY(APP_ID),
    UNIQUE (APP_ID)
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [5]:
%%sql
COPY Google_Playstore FROM '/home/ubuntu/Google_Playstore.csv' DELIMITER ',';

 * postgresql://student@/Group1
266994 rows affected.


[]

## Facts and Dimensions

### We divided the original dataset 6 dimensions: Price, Category, Install,Size, Day and Evaluation; 1 fact: App.
- As in each dimention, there is no coloumn containing unique values as a natural key, thus we manually created surrogate keys.

In [6]:
from IPython.display import Image

In [7]:
Image(url="https://gwu-mileypiao888.s3.amazonaws.com/Facts.jpeg")

## Create Price Table

Price table contains information about price of apps and whether they are paid ones or free ones.

In [8]:
%%sql
DROP TABLE IF EXISTS price CASCADE;

CREATE TABLE price (
    key SERIAL PRIMARY KEY,
    price_number NUMERIC(5,2),
    is_paid NUMERIC(1,0) DEFAULT 0
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [9]:
%%sql
INSERT INTO price (price_number)
SELECT price
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [10]:
%%sql
UPDATE price
SET is_paid = 1
where price_number > 0;

 * postgresql://student@/Group1
11604 rows affected.


[]

In [11]:
%%sql
select * from price 
order by key
limit 10;

 * postgresql://student@/Group1
10 rows affected.


key,price_number,is_paid
1,0.00,0
2,0.00,0
3,0.00,0
4,0.00,0
5,5.99,1
6,0.00,0
7,0.00,0
8,0.00,0
9,0.00,0
10,0.00,0


## Create Size Table

Size table is about size information of apps.

In [12]:
%%sql
DROP TABLE IF EXISTS size CASCADE;

CREATE TABLE size (
    key SERIAL PRIMARY KEY,
    full_size VARCHAR(10),
    size_magnitude VARCHAR(2),
    size_number NUMERIC(10,2) DEFAULT 0
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [13]:
%%sql
INSERT INTO size (full_size,size_magnitude,size_number)
SELECT full_size,size_magnitude,size_number
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [14]:
%%sql
SELECT * FROM size
Limit 10;

 * postgresql://student@/Group1
10 rows affected.


key,full_size,size_magnitude,size_number
1,0M,M,0.00
2,0M,M,0.00
3,1.4M,M,1.40
4,16M,M,16.00
5,0M,M,0.00
6,23M,M,23.00
7,0M,M,0.00
8,0M,M,0.00
9,4.1M,M,4.10
10,0M,M,0.00


## Create Evaluation Table

We create Evaluation table to document the rating and review of apps. In order to create the last two columns,rating_is_top1percent_by_cate and review_is_top1percent_by_cate, we use seven temporary tables.

The temp table stores key, category_name, rating_rule, rating_number and review_number. The last two columns, rating_is_top1percent_by_cate and review_is_top1percent_by_cate, are not changed in this table.

In [15]:
%%sql
DROP TABLE IF EXISTS temp CASCADE;

CREATE TABLE temp(
    key SERIAL PRIMARY KEY,
    category_name VARCHAR(50),
    rating_rule VARCHAR(25),
    rating_number NUMERIC(3,2) DEFAULT 0,
    review_number NUMERIC(25) DEFAULT 0,
    avg_rating_by_cate NUMERIC(3,2) DEFAULT 0,
    rating_is_top1percent_by_cate BOOLEAN DEFAULT '0',
    review_is_top1percent_by_cate BOOLEAN DEFAULT '0'
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [16]:
%%sql
INSERT INTO temp (category_name, rating_number,rating_rule,review_number)
SELECT cate_name, rating, content_rating,review
FROM Google_Playstore
;

 * postgresql://student@/Group1
266994 rows affected.


[]

Temp1 table stores the column avg_rating_by_cate

In [17]:
%%sql
DROP TABLE IF EXISTS temp1;

CREATE TABLE temp1 AS
SELECT key AS key1, AVG(rating_number) OVER(PARTITION BY category_name) AS avg_rating_by_cate1
FROM temp;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

In [18]:
%%sql
SELECT * 
FROM temp1
LIMIT 5;

 * postgresql://student@/Group1
5 rows affected.


key1,avg_rating_by_cate1
308,4.2292093287827076
257177,4.2292093287827076
708,4.2292093287827076
257576,4.2292093287827076
2552,4.2292093287827076


Temp2 table stores changes in column rating_is_top1percent_by_cate.

In [19]:
%%sql
DROP TABLE IF EXISTS temp2;

CREATE TABLE temp2 AS
select key AS key2
from (
  select key, category_name, rating_rule, rating_number, review_number,
         count(*) OVER (partition by category_name) as count,
         ROW_NUMBER() OVER (partition by category_name) as row_num
  from temp
) d
where cast(cast(row_num as float)/cast(count as float) as decimal(10,3))  <= 0.01;

 * postgresql://student@/Group1
Done.
2780 rows affected.


[]

In [20]:
%%sql
ALTER TABLE temp2
ADD COLUMN rating_is_top1percent_by_cate1 BOOLEAN DEFAULT '1';

 * postgresql://student@/Group1
Done.


[]

In [21]:
%%sql
select * from temp2
limit 5;

 * postgresql://student@/Group1
5 rows affected.


key2,rating_is_top1percent_by_cate1
308,True
244750,True
708,True
259293,True
2552,True


Similarly, temp3 table stores changes in column review_is_top1percent_by_cate.

In [22]:
%%sql
DROP TABLE IF EXISTS temp3;

CREATE TABLE temp3 AS
select key AS key3
from (
  select key, category_name, rating_rule, rating_number, review_number,
         count(*) OVER (partition by category_name) as count,
         ROW_NUMBER() OVER (partition by category_name) as row_num
  from temp
) d
where cast(cast(row_num as float)/cast(count as float) as decimal(10,3))  <= 0.01;

 * postgresql://student@/Group1
Done.
2780 rows affected.


[]

In [23]:
%%sql
ALTER TABLE temp3
ADD COLUMN review_is_top1percent_by_cate1 BOOLEAN DEFAULT '1';

 * postgresql://student@/Group1
Done.


[]

In [24]:
%%sql
select * from temp3
limit 5;

 * postgresql://student@/Group1
5 rows affected.


key3,review_is_top1percent_by_cate1
308,True
244750,True
708,True
259293,True
2552,True


Temp4 table stores the join table of temp and temp. We use it to update column rating_is_top1percent_by_cate in the final Evaluation table.

In [25]:
%%sql
DROP TABLE IF EXISTS temp4;

CREATE TABLE temp4 AS
SELECT *
FROM temp
LEFT JOIN temp2
ON temp.key = temp2.key2;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

Temp5 table stores the join table of temp4 and temp3. We use it to update column review_is_top1percent_by_cate in the final Evaluation table.

In [26]:
%%sql
DROP TABLE IF EXISTS temp5;

CREATE TABLE temp5 AS
SELECT *
FROM temp4
LEFT JOIN temp3
ON temp4.key = temp3.key3;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

Temp6 table stores the join table of temp5 and temp1. We use it to update column avg_rating_by_cate in the final Evaluation table.

In [27]:
%%sql
DROP TABLE IF EXISTS temp6;

CREATE TABLE temp6 AS
SELECT *
FROM temp5
LEFT JOIN temp1
ON temp5.key = temp1.key1;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

In [28]:
%%sql
select * from temp6 limit 10

 * postgresql://student@/Group1
10 rows affected.


key,category_name,rating_rule,rating_number,review_number,avg_rating_by_cate,rating_is_top1percent_by_cate,review_is_top1percent_by_cate,key2,rating_is_top1percent_by_cate1,key3,review_is_top1percent_by_cate1,key1,avg_rating_by_cate1
7,TRAVEL_AND_LOCAL,Teen,4.27,421918,0.00,False,False,None,None,None,None,7,4.1300331026181162
13,LIFESTYLE,Everyone,3.21,754,0.00,False,False,None,None,None,None,13,4.2964418279784474
18,FOOD_AND_DRINK,Everyone,4.61,24291,0.00,False,False,None,None,None,None,18,4.3156971608832808
31,FOOD_AND_DRINK,Everyone,3.63,262042,0.00,False,False,None,None,None,None,31,4.3156971608832808
39,FOOD_AND_DRINK,Teen,4.07,485460,0.00,False,False,None,None,None,None,39,4.3156971608832808
55,GAME_ADVENTURE,Everyone 10+,3.84,3918,0.00,False,False,None,None,None,None,55,4.3261516587677725
73,GAME_PUZZLE,Everyone,4.39,10575,0.00,False,False,None,None,None,None,73,4.3596767955801105
79,GAME_ROLE_PLAYING,Mature 17+,4.19,1715,0.00,False,False,None,None,None,None,79,4.3315845410628019
95,GAME_CASUAL,Everyone,4.54,6999,0.00,False,False,None,None,None,None,95,4.2455770816158285
108,FINANCE,Everyone,4.52,50960,0.00,False,False,None,None,None,None,108,4.0418617021276596


In [29]:
%%sql
UPDATE temp6
SET rating_is_top1percent_by_cate = '1'
where rating_is_top1percent_by_cate1 = '1';

 * postgresql://student@/Group1
2780 rows affected.


[]

In [30]:
%%sql
UPDATE temp6
SET review_is_top1percent_by_cate = '1'
where review_is_top1percent_by_cate1 = '1';


 * postgresql://student@/Group1
2780 rows affected.


[]

In [31]:
%%sql
UPDATE temp6
SET avg_rating_by_cate = avg_rating_by_cate1;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [32]:
%%sql
DROP TABLE IF EXISTS evaluation CASCADE;

CREATE TABLE evaluation AS
SELECT key, rating_rule, rating_number, review_number, avg_rating_by_cate, rating_is_top1percent_by_cate, review_is_top1percent_by_cate
FROM temp6
order by key;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

In [33]:
%%sql
select * from evaluation limit 10

 * postgresql://student@/Group1
10 rows affected.


key,rating_rule,rating_number,review_number,avg_rating_by_cate,rating_is_top1percent_by_cate,review_is_top1percent_by_cate
1,Everyone,4.55,305034,4.32,False,False
2,Everyone,4.40,1207922,4.13,False,False
3,Everyone,3.66,1967,4.20,False,False
4,Everyone,4.11,389154,4.32,False,False
5,Everyone,4.65,2291,4.32,False,False
6,Everyone,3.83,2559,4.32,False,False
7,Teen,4.27,421918,4.13,False,False
8,Everyone,4.50,142618,4.32,False,False
9,Everyone,3.81,85,4.13,False,False
10,Everyone,4.55,67514,4.32,False,False


In [34]:
%%sql
DROP TABLE IF EXISTS temp;
DROP TABLE IF EXISTS temp1;
DROP TABLE IF EXISTS temp2;
DROP TABLE IF EXISTS temp3;
DROP TABLE IF EXISTS temp4;
DROP TABLE IF EXISTS temp5;
DROP TABLE IF EXISTS temp6;

 * postgresql://student@/Group1
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Create Day Table

Day table stores the date of last update information of apps.

In [35]:
%%sql
DROP TABLE IF EXISTS day CASCADE;

CREATE TABLE day (
    key SERIAL PRIMARY KEY,
    full_date VARCHAR(11),
    last_update_day VARCHAR(3),
    last_update_month VARCHAR(4),
    last_update_quarter NUMERIC(1,0),
    last_update_year VARCHAR(4)
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [36]:
%%sql
INSERT INTO day (full_date,last_update_day,last_update_month,last_update_year)
SELECT full_date,last_update_day,last_update_month,last_update_year
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

Setting column last_update_quarter.

In [37]:
%%sql
UPDATE day
SET last_update_quarter = (CASE 
WHEN last_update_month IN ('Jan','Feb','Mar') THEN 1
WHEN last_update_month IN ('Apr','May','Jun') THEN 2
WHEN last_update_month IN ('Jul','Aug','Sep') THEN 3
WHEN last_update_month IN ('Oct','Nov','Dec') THEN 4
END);

 * postgresql://student@/Group1
266994 rows affected.


[]

## Create Install Table

Install tables stores the information about install numbers and whether a certain app is in top 1 percnet in its category. We create three temporary tables in order to create the column is_top1percent_by_cate. It is the same strategy as the one we use to create Evaluation table.

First we create a temporary table to help create the Install table.

In [38]:
%%sql
DROP TABLE IF EXISTS temp;

CREATE TABLE temp (
    key SERIAL PRIMARY KEY,
    category_name VARCHAR(50),
    install_number NUMERIC(25),
    is_top1percent_by_cate BOOLEAN DEFAULT '0'
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [39]:
%%sql
INSERT INTO temp (category_name, install_number)
SELECT cate_name, install
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [40]:
%%sql
DROP TABLE IF EXISTS temp1;

CREATE TABLE temp1 AS
select key AS key1, category_name AS category_name1, install_number AS install_number1
from (
  select key, category_name, install_number,
         count(*) OVER (partition by category_name) as count,
         ROW_NUMBER() OVER (partition by category_name ORDER BY install_number DESC) as row_num
  from temp
) d
where cast(cast(row_num as float)/cast(count as float) as decimal(10,3))  <= 0.01;

 * postgresql://student@/Group1
Done.
2780 rows affected.


[]

In [41]:
%%sql
ALTER TABLE temp1
ADD COLUMN is_top1percent_by_cate1 BOOLEAN DEFAULT '1';

 * postgresql://student@/Group1
Done.


[]

In [42]:
%%sql
DROP TABLE IF EXISTS temp2;

CREATE TABLE temp2 AS
SELECT *
FROM temp
LEFT JOIN temp1
ON temp.key = temp1.key1;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

In [43]:
%%sql
UPDATE temp2
SET is_top1percent_by_cate = '1'
where is_top1percent_by_cate1 = '1';

 * postgresql://student@/Group1
2780 rows affected.


[]

In [44]:
%%sql
DROP TABLE IF EXISTS install;

CREATE TABLE install AS
SELECT key, install_number, is_top1percent_by_cate
FROM temp2;

 * postgresql://student@/Group1
Done.
266994 rows affected.


[]

In [45]:
%%sql
SELECT * from install
ORDER BY key 
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


key,install_number,is_top1percent_by_cate
1,5000000,False
2,100000000,True
3,100000,False
4,10000000,True
5,10000,False
6,100000,False
7,10000000,True
8,5000000,True
9,10000,False
10,5000000,False


In [46]:
%%sql
DROP TABLE IF EXISTS temp;
DROP TABLE IF EXISTS temp1;
DROP TABLE IF EXISTS temp2;

 * postgresql://student@/Group1
Done.
Done.
Done.


[]

## Create Category table

Category table stores the category information of apps.

In [47]:
%%sql
DROP TABLE IF EXISTS category CASCADE;

CREATE TABLE category (
    key SERIAL PRIMARY KEY,
    category_name VARCHAR(50)
);

 * postgresql://student@/Group1
Done.
Done.


[]

In [48]:
%%sql
INSERT INTO category (category_name)
SELECT cate_name
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [49]:
%%sql
SELECT *
FROM category
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


key,category_name
1,FOOD_AND_DRINK
2,TRAVEL_AND_LOCAL
3,SHOPPING
4,FOOD_AND_DRINK
5,FOOD_AND_DRINK
6,FOOD_AND_DRINK
7,TRAVEL_AND_LOCAL
8,FOOD_AND_DRINK
9,TRAVEL_AND_LOCAL
10,FOOD_AND_DRINK


## create APP table

App table is the fact table that links all the dimension tables.

In [50]:
%%sql
DROP TABLE IF EXISTS app CASCADE;

CREATE TABLE app(
    app_id NUMERIC(10) NOT NULL,
    app_name VARCHAR(100),
    price_key NUMERIC(10),
    size_key NUMERIC(10),
    install_key NUMERIC(10),
    evaluation_key NUMERIC(10),
    day_key NUMERIC(10),
    category_key NUMERIC(10),
    PRIMARY KEY(app_id),
    UNIQUE (app_id)
);


 * postgresql://student@/Group1
Done.
Done.


[]

In [51]:
%%sql
INSERT INTO App (app_id,APP_NAME)
SELECT app_id,APP_NAME
FROM Google_Playstore;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [52]:
%%sql
UPDATE App
SET price_key = app_id,
size_key = app_id,
install_key = app_id,
evaluation_key = app_id,
day_key = app_id,
category_key = app_id;

 * postgresql://student@/Group1
266994 rows affected.


[]

In [53]:
%%sql
SELECT *
FROM App
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


app_id,app_name,price_key,size_key,install_key,evaluation_key,day_key,category_key
1,DoorDash - Food Delivery,1,1,1,1,1,1
2,TripAdvisor Hotels Flights Restaurants Attractions,2,2,2,2,2,2
3,Peapod,3,3,3,3,3,3
4,foodpanda - Local Food Delivery,4,4,4,4,4,4
5,My CookBook Pro (Ad Free),5,5,5,5,5,5
6,Safeway Online Shopping,6,6,6,6,6,6
7,Yelp: Food- Shopping- Services Nearby,7,7,7,7,7,7
8,Talabat: Food Delivery,8,8,8,8,8,8
9,Fast Food Locator | Worldwide Fast Food Finder,9,9,9,9,9,9
10,Allrecipes Dinner Spinner,10,10,10,10,10,10


## Problem 1
### What is the minimum rating value of an application to be competitive in its category in the Google Play store? In which categories are the applications under the most competitive pressure?


In [54]:
%%sql
DROP VIEW IF EXISTS problem1 CASCADE;

CREATE VIEW problem1 AS

SELECT category_name, avg_rating_by_cate
FROM Category, evaluation
WHERE Category.key = evaluation.key;

 * postgresql://student@/Group1
Done.
Done.


[]

In [55]:
%%sql
SELECT category_name,count(category_name),avg_rating_by_cate
FROM problem1
GROUP BY category_name,avg_rating_by_cate
ORDER BY count DESC
LIMIT 15;

 * postgresql://student@/Group1
15 rows affected.


category_name,count,avg_rating_by_cate
EDUCATION,33391,4.35
TOOLS,21587,4.14
BOOKS_AND_REFERENCE,21373,4.48
ENTERTAINMENT,20603,4.26
MUSIC_AND_AUDIO,17874,4.43
LIFESTYLE,15033,4.30
PERSONALIZATION,10543,4.44
FINANCE,10340,4.04
BUSINESS,10229,4.14
PRODUCTIVITY,9141,4.17


## Problem 2
### Should app developers limit the size of an application when taking rating and installation numbers into consideration?
### What is the size of top 1% application in each category based on installation number or rating number?


In [56]:
%%sql
DROP VIEW IF EXISTS problem2 CASCADE;

CREATE VIEW problem2 AS
SELECT app_id, app_name, category_name , full_size, size_number, install_number, is_top1percent_by_cate, rating_number,rating_is_top1percent_by_cate
FROM app
JOIN size
ON app.size_key = size.key
JOIN category
ON app.category_key = category.key
JOIN install
ON app.install_key = install.key
JOIN evaluation
ON app.evaluation_key = evaluation.key
WHERE size.size_magnitude = 'M'
ORDER BY size_number DESC, category_name

 * postgresql://student@/Group1
Done.
Done.


[]

Find largest applications in each category:

In [57]:
%%sql
SELECT app_id, app_name, category_name , full_size, install_number,rating_number
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, install_number, rating_number,ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY size_number DESC ) AS RowNumberRank
        FROM problem2 
        ) d WHERE RowNumberRank <=5
order by category_name
limit 20;

 * postgresql://student@/Group1
20 rows affected.


app_id,app_name,category_name,full_size,install_number,rating_number
220719,Dreamhouse Next Launcher Theme,ART_AND_DESIGN,94M,1000,4.75
64174,Regroup AlertManager,ART_AND_DESIGN,94M,500,5.00
203658,LUBE VR,ART_AND_DESIGN,91M,1000,2.67
108152,Learn German from scratch,ART_AND_DESIGN,87M,50000,3.93
227788,Text on Photo/Image : Pic Master,ART_AND_DESIGN,81M,500,5.00
152814,Channel Tracker - YouTube client,AUTO_AND_VEHICLES,101M,1000000,3.14
162665,BALIPOST.com Online,AUTO_AND_VEHICLES,98M,1000,3.44
119902,Sudoku Wizard,AUTO_AND_VEHICLES,98M,50000,3.46
38601,Share Tube,AUTO_AND_VEHICLES,97M,10000,4.60
59771,HuHot Rewards,AUTO_AND_VEHICLES,97M,100,4.82


Find most installed application in each category:

In [58]:
%%sql
SELECT app_id, app_name, category_name , full_size, install_number,rating_number
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, install_number, rating_number,ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY install_number DESC ) AS RowNumberRank
        FROM problem2 
        ) d WHERE RowNumberRank <=1
order by category_name
limit 20;

 * postgresql://student@/Group1
20 rows affected.


app_id,app_name,category_name,full_size,install_number,rating_number
3660,Decisions: Choose Your Interactive Stories Choice,ART_AND_DESIGN,34M,100000000,4.49
20250,Houston Fire: EMS Protocols,AUTO_AND_VEHICLES,48M,10000000,4.46
27133,Epic Sax Guy Button,BEAUTY,0M,10000000,4.42
1656,Dr. Driving,BOOKS_AND_REFERENCE,0M,100000000,4.16
417,OfficeSuite - Free Office- PDF- Word-Sheets-Slides,BUSINESS,37M,100000000,4.33
3372,Google Assistant,COMICS,46M,10000000,4.18
2729,Mail1Click - Secure Mail,COMMUNICATION,0M,1000000000,4.35
1439,Hot or Not - Find someone right now,DATING,0M,10000000,4.07
2352,LEGO___ MINDSTORMS___ Commander,EDUCATION,0M,100000000,4.72
6799,TweetCaster Pro for Twitter,ENTERTAINMENT,0M,1000000000,4.30


Find the size of top5 most installed application in each category:

In [59]:
%%sql
SELECT app_id, app_name, category_name , full_size, install_number
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, install_number,ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY install_number DESC ) AS RowNumberRank
        FROM problem2
        WHERE is_top1percent_by_cate = '1'
        ) d WHERE RowNumberRank<=5
order by category_name, install_number DESC
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


app_id,app_name,category_name,full_size,install_number
3660,Decisions: Choose Your Interactive Stories Choice,ART_AND_DESIGN,34M,100000000
38313,????? MonokuroWorld,ART_AND_DESIGN,26M,50000000
715,PaperColor : Paint Draw Sketchbook & PaperDraw,ART_AND_DESIGN,16M,10000000
824,Canva: Graphic Design & Logo- Flyer- Poster maker,ART_AND_DESIGN,17M,10000000
717,ibis Paint X,ART_AND_DESIGN,30M,10000000
2255,Extreme City Mega GT Ramp 2019,AUTO_AND_VEHICLES,46M,10000000
3784,Learn professions,AUTO_AND_VEHICLES,29M,10000000
6197,???????????? ????????? - Mobile Tips 2019,AUTO_AND_VEHICLES,14M,10000000
21980,Auto Optimizer,AUTO_AND_VEHICLES,44M,10000000
115129,Calculator andanCalc LT,AUTO_AND_VEHICLES,9.3M,10000000


In [60]:
%%sql
DROP VIEW IF EXISTS install1 CASCADE;

CREATE VIEW install1 AS
SELECT app_id, app_name, category_name , full_size,size_number,install_number
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, install_number,ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY install_number DESC ) AS RowNumberRank
        FROM problem2
        WHERE is_top1percent_by_cate = '1'
        ) d WHERE RowNumberRank<=5
order by category_name, install_number DESC

 * postgresql://student@/Group1
Done.
Done.


[]

In [61]:
%%sql
SELECT category_name, AVG(size_number) FROM install1 GROUP BY category_name
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


category_name,avg
ART_AND_DESIGN,24.6000000000000000
AUTO_AND_VEHICLES,28.4600000000000000
BEAUTY,21.2000000000000000
BOOKS_AND_REFERENCE,4.3600000000000000
BUSINESS,20.3800000000000000
COMICS,19.2000000000000000
COMMUNICATION,4.0000000000000000
DATING,10.3333333333333333
EDUCATION,1.6400000000000000
ENTERTAINMENT,26.4000000000000000


Find the size of top5 rated applications in each category:

In [62]:
%%sql
SELECT app_id, app_name, category_name , full_size, rating_number 
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, rating_number, ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY rating_number DESC) AS rank
        FROM problem2
        WHERE rating_is_top1percent_by_cate = '1'
        ) d WHERE rank <= 5
order by category_name
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


app_id,app_name,category_name,full_size,rating_number
253326,Smartmilk,ART_AND_DESIGN,4M,5.00
259293,? Diamond Color by Number - Painting Games ?,ART_AND_DESIGN,6M,5.00
260055,Wrought Iron Furniture Design,ART_AND_DESIGN,8.7M,5.00
250775,Espace Maison,ART_AND_DESIGN,14M,5.00
246295,Kurabiye Tarifleri,ART_AND_DESIGN,4.1M,4.65
142012,Glooko Kiosk - Providers Only,AUTO_AND_VEHICLES,5.4M,5.00
238547,HK_Sky Line,AUTO_AND_VEHICLES,5.3M,4.73
204866,Snow Excavator Dredge Simulator - Rescue Game,AUTO_AND_VEHICLES,50M,4.66
204857,Peco Online - Preturi benzina- motorina si GPL,AUTO_AND_VEHICLES,7.2M,4.63
157462,Dominican Radio,AUTO_AND_VEHICLES,11M,4.57


In [63]:
%%sql
DROP VIEW IF EXISTS rating1 CASCADE;

CREATE VIEW rating1 AS
SELECT app_id, app_name, category_name , full_size, size_number, rating_number 
    FROM (
        SELECT app_id, app_name, category_name, full_size, size_number, rating_number, ROW_NUMBER()
          over (Partition BY category_name
                ORDER BY rating_number DESC) AS rank
        FROM problem2
        WHERE rating_is_top1percent_by_cate = '1'
        ) d WHERE rank <= 5
order by category_name

 * postgresql://student@/Group1
Done.
Done.


[]

In [64]:
%%sql
SELECT category_name, AVG(size_number) FROM rating1 GROUP BY category_name
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


category_name,avg
ART_AND_DESIGN,7.3600000000000000
AUTO_AND_VEHICLES,15.7800000000000000
BEAUTY,12.9800000000000000
BOOKS_AND_REFERENCE,8.6200000000000000
BUSINESS,9.5600000000000000
COMICS,26.9000000000000000
COMMUNICATION,14.4000000000000000
DATING,19.0333333333333333
EDUCATION,7.5000000000000000
ENTERTAINMENT,6.9000000000000000


## Problem 3
### 1) Which category has the most paid applications?

In [65]:
%%sql
DROP VIEW IF EXISTS problem3 CASCADE;

CREATE VIEW problem3 AS

SELECT category_name,price_number,is_paid
FROM Category,Price
WHERE Category.key = Price.key;

 * postgresql://student@/Group1
Done.
Done.


[]

In [66]:
%%sql
SELECT*
FROM problem3
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


category_name,price_number,is_paid
FOOD_AND_DRINK,0.00,0
TRAVEL_AND_LOCAL,0.00,0
SHOPPING,0.00,0
FOOD_AND_DRINK,0.00,0
FOOD_AND_DRINK,5.99,1
FOOD_AND_DRINK,0.00,0
TRAVEL_AND_LOCAL,0.00,0
FOOD_AND_DRINK,0.00,0
TRAVEL_AND_LOCAL,0.00,0
FOOD_AND_DRINK,0.00,0


In [67]:
%%sql
SELECT category_name,count(category_name),
SUM(CASE WHEN is_paid =1 then 1
    ELSE 0 END) AS "Paid",
SUM(CASE WHEN is_paid =0 then 1
   ELSE 0 END) AS "Free"

FROM problem3
GROUP BY category_name
ORDER BY count DESC;

 * postgresql://student@/Group1
50 rows affected.


category_name,count,Paid,Free
EDUCATION,33391,1366,32025
TOOLS,21587,1364,20223
BOOKS_AND_REFERENCE,21373,715,20658
ENTERTAINMENT,20603,370,20233
MUSIC_AND_AUDIO,17874,384,17490
LIFESTYLE,15033,340,14693
PERSONALIZATION,10543,774,9769
FINANCE,10340,140,10200
BUSINESS,10229,146,10083
PRODUCTIVITY,9141,555,8586


## 2)     Are the rating scores of paid applications higher than the average rating of unpaid applications in their categories?


In [70]:
%%sql
DROP VIEW IF EXISTS problem3_2 CASCADE;

CREATE VIEW problem3_2 AS

SELECT category_name, is_paid,AVG(rating_number) OVER(PARTITION BY category_name) AS avg_rating_by_cate
FROM Category,Price,evaluation
WHERE Category.key = Price.key AND Category.key  = evaluation.key AND is_paid =1;

 * postgresql://student@/Group1
Done.
Done.


[]

In [72]:
%%sql
SELECT category_name,count(category_name),avg_rating_by_cate
FROM problem3_2
GROUP BY category_name, avg_rating_by_cate
ORDER BY count DESC;

 * postgresql://student@/Group1
49 rows affected.


category_name,count,avg_rating_by_cate
EDUCATION,1366,4.3492386530014641
TOOLS,1364,4.3012536656891496
PERSONALIZATION,774,4.4833204134366925
BOOKS_AND_REFERENCE,715,4.3752447552447552
PRODUCTIVITY,555,4.2967567567567568
HEALTH_AND_FITNESS,425,4.2797411764705882
MEDICAL,387,4.2555297157622739
MUSIC_AND_AUDIO,384,4.3306770833333333
GAME_PUZZLE,371,4.4561725067385445
ENTERTAINMENT,370,4.2864324324324324


In [73]:
%%sql
DROP VIEW IF EXISTS problem3_3 CASCADE;

CREATE VIEW problem3_3 AS

SELECT category_name, is_paid,AVG(rating_number) OVER(PARTITION BY category_name) AS avg_rating
FROM Category,Price,evaluation
WHERE Category.key = Price.key AND Category.key  = evaluation.key AND is_paid =0;

 * postgresql://student@/Group1
Done.
Done.


[]

In [74]:
%%sql
SELECT category_name,count(category_name),avg_rating
FROM problem3_3
GROUP BY category_name,avg_rating
ORDER BY count DESC;

 * postgresql://student@/Group1
50 rows affected.


category_name,count,avg_rating
EDUCATION,32025,4.3471875097580016
BOOKS_AND_REFERENCE,20658,4.4793295575563946
ENTERTAINMENT,20233,4.2570202145010626
TOOLS,20223,4.1315729614795035
MUSIC_AND_AUDIO,17490,4.4331932532875929
LIFESTYLE,14693,4.2978302593071531
FINANCE,10200,4.0374362745098039
BUSINESS,10083,4.1419289893880789
PERSONALIZATION,9769,4.4315733442522264
PRODUCTIVITY,8586,4.1642080130444910


## 3)     What is the relationship between price and installations of applications in their categories? Are they professional applications?

In [75]:
%%sql
DROP VIEW IF EXISTS relation CASCADE;

CREATE VIEW relation AS

SELECT price_number,install_number
FROM Price, install
WHERE install.key = Price.key;

 * postgresql://student@/Group1
Done.
Done.


[]

In [76]:
%%sql
SELECT install_number,count(install_number),AVG(price_number) OVER(PARTITION BY install_number) AS avg_price_number
FROM relation
GROUP BY install_number,price_number
ORDER BY install_number DESC
LIMIT 10;

 * postgresql://student@/Group1
10 rows affected.


install_number,count,avg_price_number
5000000000,3,0E-20
1000000000,24,0E-20
500000000,35,0E-20
100000000,283,0E-20
50000000,414,0E-20
10000000,1,2.6600000000000000
10000000,2819,2.6600000000000000
10000000,1,2.6600000000000000
5000000,1,0.49500000000000000000
5000000,2803,0.49500000000000000000


## 4)     How many applications with top 1% installations are paid ones in a certain category? Choose several categories with largest number of paid installations and do the analysis.

In [77]:
%%sql
DROP VIEW IF EXISTS top1 CASCADE;

CREATE VIEW top1 AS

SELECT is_paid,is_top1percent_by_cate
FROM Price, install
WHERE install.key = Price.key;

 * postgresql://student@/Group1
Done.
Done.


[]

In [78]:
%%sql
SELECT count(is_top1percent_by_cate),
SUM(CASE WHEN is_paid =1 then 1
    ELSE 0 END) AS "Paid",
SUM(CASE WHEN is_paid =0 then 1
   ELSE 0 END) AS "Free"

FROM top1
WHERE is_top1percent_by_cate ='True';

 * postgresql://student@/Group1
1 rows affected.


count,Paid,Free
2780,0,2780
